In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

### Connecting to Postgres 

In order to set up the PostGres Database, we used docker-compose to build the jupyter and postgres containers. However, we run into the issue with data persistence as when the PostGres container goes down, the data within that container gets removed. And so we need to create a volume that houses all our data so that whenever our container goes down, the volume will hold all of it. And when the container comes back up, the volume will be operating in the container.

The diagram below shows the setup for this. Also running is our Jupyter notebook container for which this current notebook is running on.

![postgres](https://git.generalassemb.ly/raw/michaelkim9/semantic_search/master/other_assets/docker_postgres_setup.png)

In [2]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor

def execute_query(query):  
    connection = pg2.connect(host='postgres',
                             user='postgres',
                             database='wikipedia')
    cursor = connection.cursor(cursor_factory=RealDictCursor)
    cursor.execute(query)
    r = cursor.fetchall()
    cursor.close()
    return r

def query_to_df(query):
    df = pd.DataFrame(execute_query(query))
    return df


### EDA on the tables
Doing some initial investigation of the tables. Will need to do some data cleaning below

##### Categories Table

In [3]:
categories = query_to_df('SELECT * FROM categories;')
categories.head()

,category_id,category_name,category_summary,url
0,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...
1,33547387,Category:Applied machine learning,None,https://en.wikipedia.org/wiki/Category:Applied...
2,42936114,Category:Artificial neural networks,This category are for articles about artificia...,https://en.wikipedia.org/wiki/Category:Artific...
3,49119651,Category:Deep learning,None,https://en.wikipedia.org/wiki/Category:Deep_le...
4,3735046,Category:Neural network software,This category lists Artificial neural network ...,https://en.wikipedia.org/wiki/Category:Neural_...


In [4]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 4 columns):
category_id         208 non-null int64
category_name       208 non-null object
category_summary    72 non-null object
url                 208 non-null object
dtypes: int64(1), object(3)
memory usage: 6.6+ KB


In [5]:
value_dicts = execute_query('SELECT DISTINCT category_name FROM categories;')
np.array([list(i.values())[0] for i in value_dicts])[1:]

array(['Category:Financial software companies',
       'Category:Reporting software',
       'Category:Sports management video games',
       'Category:Office suites for Linux', 'Category:Wiki communities',
       'Category:Business software companies',
       'Category:Cluster analysis algorithms',
       'Category:Computational learning theory',
       'Category:Dental software', 'Category:Supervised learning',
       'Category:Time-tracking software', 'Category:Records management',
       'Category:Free static website generators',
       'Category:Business software for Windows',
       'Category:Accounting software', 'Category:Offline mail readers',
       'Category:Learning in computer vision',
       'Category:Enterprise resource planning terminology',
       'Category:Mobile business software', 'Category:WikiLeaks',
       'Category:Markov models', 'Category:Free reporting software',
       'Category:Office software', 'Category:Free TeX software',
       'Category:TeX', 'Category

##### Pages Table

In [6]:
pages = query_to_df('SELECT * FROM pages;')
pages.head()

,links,page_categories,page_id,summary,title,url
0,"['Alteryx', 'Data Visualization', 'Data profil...","['All articles lacking in-text citations', 'Al...",43385931,Data exploration is an approach similar to ini...,Data exploration,https://en.wikipedia.org/wiki/Data_exploration
1,"['Academic journal', 'Algorithm', 'Amazon.com'...","['Artificial intelligence', 'CS1 maint: Multip...",49082762,These datasets are used for machine learning r...,List of datasets for machine learning research,https://en.wikipedia.org/wiki/List_of_datasets...
2,"['ADALINE', 'AT&T Labs', 'Adaptive website', '...",['All articles containing potentially dated st...,233488,Machine learning is the subfield of computer s...,Machine learning,https://en.wikipedia.org/wiki/Machine_learning
3,"['ANOVA', 'AODE', 'Academic discipline', 'AdaB...","['Artificial intelligence', 'Data mining', 'Ma...",53587467,The following outline is provided as an overvi...,Outline of machine learning,https://en.wikipedia.org/wiki/Outline_of_machi...
4,"['Machine Learning', 'Widely Applicable Inform...","['All articles covered by WikiProject Wikify',...",53198248,"In Machine Learning, if a statistical model is...",Singular statistical model,https://en.wikipedia.org/wiki/Singular_statist...


In [7]:
pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4207 entries, 0 to 4206
Data columns (total 6 columns):
links              4207 non-null object
page_categories    4207 non-null object
page_id            4207 non-null int64
summary            4180 non-null object
title              4207 non-null object
url                4207 non-null object
dtypes: int64(1), object(5)
memory usage: 197.3+ KB


##### Categories_Pages Table

In [8]:
categories_pages = query_to_df('SELECT * FROM categories_pages;')
categories_pages.head()

,category_id,category_name,page_id,time_stamp,title,type
0,706543,Category:Machine_learning,43385931,2017-07-18 06:39:35,Data exploration,page
1,706543,Category:Machine_learning,49082762,2017-08-08 12:44:17,List of datasets for machine learning research,page
2,706543,Category:Machine_learning,233488,2015-06-29 12:07:35,Machine learning,page
3,706543,Category:Machine_learning,53587467,2017-03-25 20:23:08,Outline of machine learning,page
4,706543,Category:Machine_learning,53198248,2017-03-16 21:50:50,Singular statistical model,page


In [9]:
categories_pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6443 entries, 0 to 6442
Data columns (total 6 columns):
category_id      6443 non-null int64
category_name    6443 non-null object
page_id          6443 non-null int64
time_stamp       6443 non-null datetime64[ns]
title            6443 non-null object
type             6443 non-null object
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 302.1+ KB


### DataFrame for NLP and Vectorization

For the the purpose of performing NLP and more specifically LSA, we won't need all the columns - just the relevant information. Creating a working dataframe for the purposes of our analysis.

Once we create this dataframe, then we'll need to do some data cleaning, mainly dealing with the null values.

It looks like the null values are primarily in the summary columns of the categories and pages table. The summary for the categories table won't matter so much as it doesn't directly affect our analysis and model. However, our entire analysis is primarily done on the summary column of the pages table and this column has some null values. The pages that have no summaries indicate that the specific Wikipedia page doesn't have any content (maybe there are some tables and links but no actual information on the title topic). Therefore, I am planning to just remove rows that have null page summaries as they don't do anything towards training our model that is all built upon analyzing text.

After the dataframe is cleaned up, going to create another column that combines the text of the page title and page summary as 'page_text' which will be the text we vectorize for NLP.

In [10]:
df = query_to_df("""
SELECT cp.page_id, cp.title as page_title, p.summary as page_summary, p.url as page_url,
    cp.category_id, cp.category_name, c.category_summary, c.url as category_url
FROM categories_pages cp
LEFT JOIN pages p ON cp.page_id = p.page_id
LEFT JOIN categories c ON cp.category_id = c.category_id
""")
df.head()

,category_id,category_name,category_summary,category_url,page_id,page_summary,page_title,page_url
0,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...,43385931,Data exploration is an approach similar to ini...,Data exploration,https://en.wikipedia.org/wiki/Data_exploration
1,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...,49082762,These datasets are used for machine learning r...,List of datasets for machine learning research,https://en.wikipedia.org/wiki/List_of_datasets...
2,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...,233488,Machine learning is the subfield of computer s...,Machine learning,https://en.wikipedia.org/wiki/Machine_learning
3,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...,53587467,The following outline is provided as an overvi...,Outline of machine learning,https://en.wikipedia.org/wiki/Outline_of_machi...
4,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...,53198248,"In Machine Learning, if a statistical model is...",Singular statistical model,https://en.wikipedia.org/wiki/Singular_statist...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6443 entries, 0 to 6442
Data columns (total 8 columns):
category_id         6443 non-null int64
category_name       6443 non-null object
category_summary    2845 non-null object
category_url        6443 non-null object
page_id             6443 non-null int64
page_summary        6409 non-null object
page_title          6443 non-null object
page_url            6443 non-null object
dtypes: int64(2), object(6)
memory usage: 402.8+ KB


In [12]:
df['category_summary'].fillna('No Summary', inplace=True)

In [13]:
df.drop(df[df['page_summary'].isnull()].index.tolist(),axis=0,inplace=True)

In [14]:
df['page_text'] = df['page_title'].astype(str)+' '+df['page_summary']

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6409 entries, 0 to 6442
Data columns (total 9 columns):
category_id         6409 non-null int64
category_name       6409 non-null object
category_summary    6409 non-null object
category_url        6409 non-null object
page_id             6409 non-null int64
page_summary        6409 non-null object
page_title          6409 non-null object
page_url            6409 non-null object
page_text           6409 non-null object
dtypes: int64(2), object(7)
memory usage: 500.7+ KB


In [16]:
df.head()

,category_id,category_name,category_summary,category_url,page_id,page_summary,page_title,page_url,page_text
0,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...,43385931,Data exploration is an approach similar to ini...,Data exploration,https://en.wikipedia.org/wiki/Data_exploration,Data exploration Data exploration is an approa...
1,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...,49082762,These datasets are used for machine learning r...,List of datasets for machine learning research,https://en.wikipedia.org/wiki/List_of_datasets...,List of datasets for machine learning research...
2,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...,233488,Machine learning is the subfield of computer s...,Machine learning,https://en.wikipedia.org/wiki/Machine_learning,Machine learning Machine learning is the subfi...
3,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...,53587467,The following outline is provided as an overvi...,Outline of machine learning,https://en.wikipedia.org/wiki/Outline_of_machi...,Outline of machine learning The following outl...
4,706543,Category:Machine_learning,Machine learning is a branch of statistics and...,https://en.wikipedia.org/wiki/Category:Machine...,53198248,"In Machine Learning, if a statistical model is...",Singular statistical model,https://en.wikipedia.org/wiki/Singular_statist...,Singular statistical model In Machine Learning...


### Apply cleaner to the page_text column

In order for the text to be used for NLP, we need to do some data cleaning. Defined a function below with regular expressions to clean the data.

In [18]:
import re
from spacy.en import STOP_WORDS, English
nlp = English()

In [19]:
def cleaner(text):
    text = text.lower()
    text = re.sub('{\\\\.+}','',text)
    text = re.sub('&#39;','',text)
    text = re.sub('<br />','',text)
    text = re.sub('<.*>.*</.*>','', text)
    text = re.sub('\\ufeff', '', text)
    text = re.sub('\000', '', text)
    text = re.sub('\xFF', '', text)
    text = re.sub('\uFFFF', '', text)
    text = re.sub(u'[^a-z\s]','',text)
    text = re.sub('\cI', '', text)
    text = re.sub('\t', '', text)
    text = re.sub('\f', '', text)
    text = re.sub('\r', '', text)
    text = re.sub('\(','',text)
    text = re.sub('\)','',text)
    text = re.sub('\n',' ',text)
    text = re.sub('\'a',' ',text)
    text = re.sub('\{','',text)
    text = re.sub('\}','',text)
    text = re.sub('\+','',text)
    text = re.sub('\*','',text)  
    text = re.sub('\  ', '', text)
    text = ' '.join(i.lemma_ for i in nlp(text) if i.orth_ not in STOP_WORDS)
    text = ' '.join(text.split())
    return text

In [20]:
df['clean_text'] = df['page_text'].apply(cleaner)

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:12: DeprecationWarning: bad escape \c
  if sys.path[0] == '':


In [21]:
df[['page_text','page_title','clean_text']].sample(10)

,page_text,page_title,clean_text
2659,OpenGate OpenGate is a communications platform...,OpenGate,opengate opengate communications platform desi...
3040,ClearBooks Clear Books is a cloud-based (SaaS)...,ClearBooks,clearbook clear book cloudbased saas accountin...
3094,Agorum core agorum core is a free Open-Source ...,Agorum core,agorum core agorum core free opensource enterp...
1588,"Autoencoder An autoencoder, autoassociator or ...",Autoencoder,autoencoder autoencoder autoassociator diabolo...
4246,EFileCabinet eFileCabinet is a company headqua...,EFileCabinet,efilecabinet efilecabinet company headquarter ...
2433,D3.ru Dirty.ru is one of the first collaborati...,D3.ru,dru dirtyru collaborative blog runet online pl...
5238,TeX Directory Structure In typesetting softwar...,TeX Directory Structure,tex directory structure typesetting software t...
1088,"Artificial ants In computer science, Artificia...",Artificial ants,artificial ant computer science artificial ant...
4953,Webmail Webmail (or web-based email) is any em...,Webmail,webmail webmail webbas email email client impl...
4612,WikiHow wikiHow is an online wiki-style commun...,WikiHow,wikihow wikihow online wikistyle community con...


### TFIDF LSA - Label Encode The Title

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
le = LabelEncoder()
df['page_title_numerical'] = le.fit_transform(df['page_title'])

In [24]:
df[['clean_text','page_title','page_title_numerical']].sample(5)

,clean_text,page_title,page_title_numerical
1643,computer reservation system computer reservati...,Computer reservation system,694
4250,ibm connection ibm connection webenterprise so...,IBM Connections,1626
3559,radiological information system radiological i...,Radiological information system,3053
5221,luatex luatex texbased computer typesetting sy...,LuaTeX,2070
2569,oracle enterprise resource planning cloud orac...,Oracle Enterprise Resource Planning Cloud,2682


### Prepare the Document Term Matrix - TFDIF Vectorizer

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
tfidf_vectorizer = TfidfVectorizer(min_df = 1, stop_words = 'english')

In [27]:
document_term_matrix_sps = tfidf_vectorizer.fit_transform(df['clean_text'])

In [28]:
document_term_matrix_sps

<6409x28073 sparse matrix of type '<class 'numpy.float64'>'
	with 303370 stored elements in Compressed Sparse Row format>

In [29]:
document_term_matrix_df = pd.DataFrame(document_term_matrix_sps.toarray(),
                                       index=df.index,
                                       columns=tfidf_vectorizer.get_feature_names())

### Pickle files for further analysis in next notebook

Please move onto part 2.2 notebook

In [30]:
from sklearn.externals import joblib

In [ ]:
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer')
joblib.dump(document_term_matrix_sps, 'document_term_matrix_sps')
joblib.dump(df,'df')